<h1 align="center">Eleven key measures for monitoring general practice clinical activity during COVID-19 using federated analytics on 53 million adults’ primary care records through OpenSAFELY </h1>

**This notebook accompanies [this paper (add link)](), describing trends and variation in clinical activity codes using a set of key measures indicative of overall activity to evaluate NHS service restoration throughout the COVID-19 pandemic.**

The aim of the OpenSAFELY SRO is to describe trends and variation in clinical activity codes to evaluate NHS service restoration during the COVID-19 pandemic.

The purpose of this notebook is to provide a set of key GP measures at the practice level, that are indicative of changes in overall activity during the pandemic. For each of these measures we provide a link to the codelist containing all the codes used for that measure, a description of what the measure is and a brief overview of why the measure is important. We also highlight any caveats, where there are any, for each measure.  

Monthly rates of recorded activity are displayed as practice level decile charts to show both the general trend and practice level variation in activity changes. Accompanying each chart is a summary of the most commonly recorded SNOMED codes for each measure.

For each measure we also indicate the number of unique patients recorded as having at least one event indicated by the measure as well as the total number of events since January 2019.
A summary of the number of events for each measure is produced and monthly rates of recorded activity for each measure is plotted as a decile chart.

For each measure, we also median activity rate in April 2019 (baseline) and the percentage change from this median in April 2020 (time of the 1st national lockdown) and April 2021. These changes are used to give an overall classification of activity change as described in the box below.

* No change: no change from baseline in both April 2020 and April 2021.
* Increase: an increase from baseline in either April 2020 or April 2021.
* Sustained drop: a drop from baseline of >15% in April 2020 which **has not** returned to within 15% of the baseline by April 2021.
* Recovery: a drop of >15% from baseline in April 2020 which **has** returned to within 15% of the baseline by April 2021.

The following key measures are provided:

<ul id="docNav">
    <li> <a href="#systolic_bp">Blood Pressure Monitoring</a>
    <li> <a href="#qrisk2">Cardiovascular Disease 10 Year Risk Assessment</a>
    <li> <a href="#cholesterol">Cholesterol Testing</a>
    <li> <a href="#ALT">Liver Function Testing - Alanine Transferaminase (ALT)</a>
    <li> <a href="#serum_tsh">Thyroid Testing</a>
    <li> <a href="#rbc_fbc">Full Blood Count - Red Blood Cell (RBC) Testing</a>
    <li> <a href="#hba1c">Glycated Haemoglobin A1c Level (HbA1c)</a>
    <li> <a href="#serum_sodium">Renal Function Assessment - Sodium Testing</a>
    <li> <a href="#asthma">Asthma Reviews</a>
    <li> <a href="#copd">Chronic Obstrutive Pulmonary Disease (COPD) Reviews</a>
    <li> <a href="#med_review">Medication Review</a>
</ul>


In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utilities import *
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline
%config InlineBackend.figure_format='png'


In [ ]:
def deciles_chart(
    df,
    period_column=None,
    column=None,
    title="",
    ylabel="",
    interactive=True,
    width=800,
    height=400,
):
    """period_column must be dates / datetimes"""
    
    df = compute_deciles(df, period_column, column, False)
  
    if interactive:

        fig = go.Figure()

        linestyles = {
            "decile": {"color": "blue", "dash": "dash"},
            "median": {"color": "blue", "dash": "solid"},
            "percentile": {"color": "blue", "dash": "dash"},
        }

        for percentile in np.unique(df["percentile"]):
            df_subset = df[df["percentile"] == percentile]
            if percentile == 50:
                fig.add_trace(
                    go.Scatter(
                        x=df_subset[period_column],
                        y=df_subset[column],
                        line={"color": "blue", "dash": "solid", "width": 1.2},
                        name="median",
                    )
                )
            else:
                fig.add_trace(
                    go.Scatter(
                        x=df_subset[period_column],
                        y=df_subset[column],
                        line={"color": "blue", "dash": "dash", "width": 1},
                        name=f"decile {int(percentile/10)}",
                    )
                )

        # Set title
        fig.update_layout(
            title_text=title,
            hovermode="x",
            title_x=0.5,
            width=width,
            height=height,
        )

        fig.update_yaxes(title=ylabel)
        fig.update_xaxes(title="Date")

        # Add range slider
        fig.update_layout(
            xaxis=go.layout.XAxis(
                rangeselector=dict(
                    buttons=list(
                        [
                            dict(
                                count=1,
                                label="1m",
                                step="month",
                                stepmode="backward",
                            ),
                            dict(
                                count=6,
                                label="6m",
                                step="month",
                                stepmode="backward",
                            ),
                            dict(
                                count=1,
                                label="1y",
                                step="year",
                                stepmode="backward",
                            ),
                            dict(step="all"),
                        ]
                    )
                ),
                rangeslider=dict(visible=True),
                type="date",
            )
        )

        fig.show()

    else:
        px = 1 / plt.rcParams["figure.dpi"]  # pixel in inches
        fig, ax = plt.subplots(
            1, 1, figsize=(width * px, height * px), tight_layout=True
        )

        deciles_chart_ebm(
            df,
            period_column="date",
            column="rate",
            ylabel="rate per 1000",
            show_outer_percentiles=False,
            ax=ax,
        )

In [ ]:
%%capture --no-display

sentinel_measures = ["qrisk2", "asthma", "copd", "sodium", "cholesterol", "alt", "tsh", "rbc", 'hba1c', 'systolic_bp', 'medication_review']


data_dict_practice = {}
childs_table_dict = {}



with open("../backend_outputs/emis/patient_count.json") as f:
        num_patients_emis = json.load(f)["num_patients"]

with open("../backend_outputs/emis/event_count.json") as f:
        num_events_emis = json.load(f)["num_events"]

with open("../backend_outputs/tpp/patient_count.json") as f:
        num_patients_tpp = json.load(f)["num_patients"]

with open("../backend_outputs/tpp/event_count.json") as f:
        num_events_tpp = json.load(f)["num_events"]

num_patients = {}
num_events = {}
for key, value in num_patients_emis.items():

        num_patients[key] = value + num_patients_tpp[key]

for key, value in num_events_emis.items():
        value = value/1_000_000
        num_events[key] = value + (num_events_tpp[key]/1_000_000)

for measure in sentinel_measures:
    df = pd.read_csv(f"../backend_outputs/measure_{measure}.csv", parse_dates=["date"])
    data_dict_practice[measure] = df
    child_table = pd.read_csv(f"../backend_outputs/code_table_{measure}.csv", index_col=0)

    child_table = child_table.rename(columns={"Description_tpp":"Description"})
    child_table.loc[child_table["Description"].isnull(), "Description"] = child_table.loc[child_table["Description"].isnull(), "Description_emis"]

    child_table = child_table.loc[:, ["Code","Description","Proportion of Codes (%)"]]
        
    childs_table_dict[measure] = child_table

def generate_sentinel_measure_combined(data_dict_practice, child_table, measure, num_patients, num_events, codelist_link):
        df = data_dict_practice[measure]
        display(
                Markdown(f"Rate per 1000 registered patients")
        )

        deciles_chart(
                df,
                period_column="date",
                column="rate",
                ylabel="rate per 1000",
                interactive=False,
        )

        display(
                Markdown(f"#### Most Common Codes <a href={codelist_link}>(Codelist)</a>"),
                HTML(child_table.to_html(index=False)),
        )

        display(
                Markdown(f"Total patients: {num_patients:.2f}M ({num_events:.2f}M events)")
        )
        return df

In [ ]:
def deciles_chart_ebm(
    df,
    period_column=None,
    column=None,
    title="",
    ylabel="",
    show_outer_percentiles=True,
    show_legend=True,
    ax=None,
):
    """period_column must be dates / datetimes"""
    sns.set_style("whitegrid", {"grid.color": ".9"})
    if not ax:
        fig, ax = plt.subplots(1, 1)
    df = compute_deciles(df, period_column, column, show_outer_percentiles)
    
    linestyles = {
        "decile": {
            "line": "b--",
            "linewidth": 1,
            "label": "decile",
        },
        "median": {
            "line": "b-",
            "linewidth": 1.5,
            "label": "median",
        },
        "percentile": {
            "line": "b:",
            "linewidth": 0.8,
            "label": "1st-9th, 91st-99th percentile",
        },
    }
    label_seen = []
    
    for percentile in range(1, 100):  # plot each decile line
        data = df[df["percentile"] == percentile]
        add_label = False

        if percentile == 50:
            style = linestyles["median"]
            add_label = True
        elif show_outer_percentiles and (percentile < 10 or percentile > 90):
            style = linestyles["percentile"]
            if "percentile" not in label_seen:
                label_seen.append("percentile")
                add_label = True
        else:
            style = linestyles["decile"]
            if "decile" not in label_seen:
                label_seen.append("decile")
                add_label = True
        if add_label:
            label = style["label"]
        else:
            label = "_nolegend_"

        ax.plot(
            data[period_column],
            data[column],
            style["line"],
            linewidth=style["linewidth"],
            label=label,
        )
    ax.set_ylabel(ylabel, size=15, alpha=0.6)
    if title:
        ax.set_title(title, size=18)
    # set ymax across all subplots as largest value across dataset
    ax.set_ylim([0, df[column].max() * 1.05])
    ax.tick_params(labelsize=12)
    ax.set_xlim(
        [df[period_column].min(), df[period_column].max()]
    )  # set x axis range as full date range
    
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%B %Y"))
    ax.xaxis.set_major_locator(matplotlib.dates.MonthLocator(interval=1))
    if show_legend:
        ax.legend(
            bbox_to_anchor=(1.05, 0.6),
            ncol=1,
            fontsize=12,
            borderaxespad=0.0,
            frameon=True,
        )

    # rotates and right aligns the x labels, and moves the bottom of the
    # axes up to make room for them
    plt.gcf().autofmt_xdate(rotation=90, ha="center", which="both")
    
    plt.show()
    return plt

def compute_deciles(measure_table, groupby_col, values_col, has_outer_percentiles=True):
    """Computes deciles.

    Args:
        measure_table: A measure table.
        groupby_col: The name of the column to group by.
        values_col: The name of the column for which deciles are computed.
        has_outer_percentiles: Whether to compute the nine largest and nine smallest
            percentiles as well as the deciles.

    Returns:
        A data frame with `groupby_col`, `values_col`, and `percentile` columns.
    """
    quantiles = np.arange(0.1, 1, 0.1)
    if has_outer_percentiles:
        quantiles = np.concatenate(
            [quantiles, np.arange(0.01, 0.1, 0.01), np.arange(0.91, 1, 0.01)]
        )

    percentiles = (
        measure_table.groupby(groupby_col)[values_col]
        .quantile(pd.Series(quantiles))
        .reset_index()
    )
    
    percentiles["percentile"] = percentiles["level_1"].apply(lambda x: int(x * 100))
    return percentiles

<a id="systolic_bp"></a>
### Blood Pressure Monitoring

The codes used in for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/systolic-blood-pressure-qof/3572b5fb/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

A commonly-used assessment used to identify patients with hypertension or to ensure optimal treatment for those with known hypertension.  This helps ensure appropriate treatment, with the aim of reducing long term risks of complications from hypertension such as stroke, myocardial infarction and kidney disease. 

In [ ]:
systolic_bp_df = generate_sentinel_measure_combined(data_dict_practice, childs_table_dict["systolic_bp"], "systolic_bp", num_patients["systolic_bp"], num_events["systolic_bp"], codelist_link="https://www.opencodelists.org/codelist/opensafely/systolic-blood-pressure-qof/3572b5fb/")

In [ ]:
baseline, values, differences = calculate_statistics(systolic_bp_df, '2019-04-01', ['2020-04-01', '2021-04-01'])
display_changes(baseline, values, differences, ['April 2020', 'April 2021'])
classify_changes(differences)

<a id="qrisk2"></a>
### Cardiovascular Disease 10 year Risk Assessment

The codes used in for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/cvd-risk-assessment-score-qof/1adf44a5/">this codelist</a>.

<h3 class="details">What is it and why does it matter? </h3>

A commonly-used risk assessment used to identify patients with an increased risk of cardiovascular events in the next 10 years. This helps ensure appropriate treatment, with the aim of reducing long term risks of complications such as stroke or myocardial infarction. 

In [ ]:
qrisk2_df = generate_sentinel_measure_combined(data_dict_practice, childs_table_dict["qrisk2"], "qrisk2", num_patients["qrisk2"], num_events["qrisk2"], codelist_link="https://www.opencodelists.org/codelist/opensafely/cvd-risk-assessment-score-qof/1adf44a5/")

In [ ]:
baseline, values, differences = calculate_statistics(qrisk2_df, '2019-04-01', ['2020-04-01', '2021-04-01'])
display_changes(baseline, values, differences, ['April 2020', 'April 2021'])
classify_changes(differences)

<a id="cholesterol"></a>
### Cholesterol Testing

The codes used in for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/cholesterol-tests/09896c09/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

A commonly-used blood test used as part of a routine cardiovascular disease 10 year risk assessment and also to identify patients with lipid disorders (e.g. familial hypercholesterolaemia). This helps ensure appropriate treatment, with the aim of reducing long term risks of complications such as stroke or myocardial infarction.

In [ ]:
cholesterol_df = generate_sentinel_measure_combined(data_dict_practice, childs_table_dict["cholesterol"], "cholesterol", num_patients["cholesterol"], num_events["cholesterol"], codelist_link="https://www.opencodelists.org/codelist/opensafely/cholesterol-tests/09896c09/")

In [ ]:
baseline, values, differences = calculate_statistics(cholesterol_df, '2019-04-01', ['2020-04-01', '2021-04-01'])
display_changes(baseline, values, differences, ['April 2020', 'April 2021'])
classify_changes(differences)

<a id="ALT"></a>
### Liver Function Testing - Alanine Transferaminase (ALT)

The codes used in for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/alanine-aminotransferase-alt-tests/2298df3e/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

An ALT blood test is one of a group of liver function tests (LFTs) which are used to detect problems with the function of the liver.  It is often used to monitor patients on medications which may affect the liver or which rely on the liver to break them down within the body. They are also tested for patients with known or suspected liver dysfunction.  

<h3 class="details">Caveats</h3>
**In a small number of places, an ALT test may NOT be included within a liver function test.** We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only tests results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
alt_df = generate_sentinel_measure_combined(data_dict_practice, childs_table_dict["alt"], "alt", num_patients["alt"], num_events["alt"], codelist_link="https://www.opencodelists.org/codelist/opensafely/alanine-aminotransferase-alt-tests/2298df3e/")

In [ ]:
baseline, values, differences = calculate_statistics(alt_df, '2019-04-01', ['2020-04-01', '2021-04-01'])
display_changes(baseline, values, differences, ['April 2020', 'April 2021'])
classify_changes(differences)

<a id="serum_tsh"></a>
### Thyroid Testing

The codes used in for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/thyroid-stimulating-hormone-tsh-testing/11a1abeb/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

An ALT blood test is one of a group of liver function tests (LFTs) which are used to detect problems with the function of the liver.  It is often used to monitor patients on medications which may affect the liver or which rely on the liver to break them down within the body. They are also tested for patients with known or suspected liver dysfunction.  

In [ ]:
tsh_df = generate_sentinel_measure_combined(data_dict_practice, childs_table_dict["tsh"], "tsh", num_patients["tsh"], num_events["tsh"], codelist_link="https://www.opencodelists.org/codelist/opensafely/thyroid-stimulating-hormone-tsh-testing/11a1abeb/")

In [ ]:
baseline, values, differences = calculate_statistics(tsh_df, '2019-04-01', ['2020-04-01', '2021-04-01'])
display_changes(baseline, values, differences, ['April 2020', 'April 2021'])
classify_changes(differences)

<a id="rbc_fbc"></a>
### Full Blood Count - Red Blood Cell (RBC) Testing

The codes used in for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/red-blood-cell-rbc-tests/576a859e/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

RBC is completed as part of a group of tests referred to as a full blood count (FBC), used to detect a variety of disorders of the blood, such as anaemia and infection.

<h3 class="details">Caveats</h3>
Here, we use codes which represent results reported to GPs, so tests requested but not yet reported are not included. This will usually exclude tests requested while a person is in hospital and other settings, like a private clinic.

In [ ]:
rbc_df = generate_sentinel_measure_combined(data_dict_practice, childs_table_dict["rbc"], "rbc", num_patients["rbc"], num_events["rbc"], codelist_link="https://www.opencodelists.org/codelist/opensafely/red-blood-cell-rbc-tests/576a859e/")

In [ ]:
baseline, values, differences = calculate_statistics(rbc_df, '2019-04-01', ['2020-04-01', '2021-04-01'])
display_changes(baseline, values, differences, ['April 2020', 'April 2021'])
classify_changes(differences)

<a id="hba1c"></a>
### Glycated Haemoglobin A1c Level (HbA1c)

The codes used in for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/glycated-haemoglobin-hba1c-tests/62358576/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

HbA1c is a long term indicator of diabetes control. NICE guidelines recommend that individuals with diabetes have their HbA1c measured at least twice a year. Poor diabetic control can place individuals living with diabetes at an increased risk of the complications of diabetes.

<h3 class="details">Caveats</h3>
Here, we use codes which represent results reported to GPs, so tests requested but not yet reported are not included. This will usually exclude tests requested while a person is in hospital and other settings, like a private clinic.

In [ ]:
hba1c_df = generate_sentinel_measure_combined(data_dict_practice, childs_table_dict["hba1c"], "hba1c", num_patients["hba1c"], num_events["hba1c"], codelist_link="https://www.opencodelists.org/codelist/opensafely/glycated-haemoglobin-hba1c-tests/62358576/")

In [ ]:
baseline, values, differences = calculate_statistics(hba1c_df, '2019-04-01', ['2020-04-01', '2021-04-01'])
display_changes(baseline, values, differences, ['April 2020', 'April 2021'])
classify_changes(differences)

<a id="serum_sodium"></a>
### Renal Function Assessment - Sodium Testing

The codes used in for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/sodium-tests-numerical-value/32bff605/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

Sodium is completed as part of a group of tests referred to as a renal profile, used to detect a variety of disorders of the kidneys. A renal profile is also often used to monitor patients on medications which may affect the kidneys or which rely on the kidneys to remove them from the body.

<h3 class="details">Caveats</h3>
Here, we use codes which represent results reported to GPs, so tests requested but not yet reported are not included. This will usually exclude tests requested while a person is in hospital and other settings, like a private clinic.

In [ ]:
sodium_df = generate_sentinel_measure_combined(data_dict_practice, childs_table_dict["sodium"], "sodium", num_patients["sodium"], num_events["sodium"], codelist_link="https://www.opencodelists.org/codelist/opensafely/sodium-tests-numerical-value/32bff605/")

In [ ]:
baseline, values, differences = calculate_statistics(sodium_df, '2019-04-01', ['2020-04-01', '2021-04-01'])
display_changes(baseline, values, differences, ['April 2020', 'April 2021'])
classify_changes(differences)

<a id="asthma"></a>
### Asthma Reviews

The codes used in for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/asthma-annual-review-qof/33eeb7da/">this codelist</a>.  QoF recommends a number of codes that can be used by practices as an asthma annual review.  These are all included in our codelist.

<h3 class="details">What is it and why does it matter?</h3>

The British Thoracic Society and Scottish Intercollegiate Guidelines Network on the management of asthma recommend that people with asthma receive a review of their condition at least annually.If a patient has not been reviewed, it is possible that their asthma control may have worsened, leading to a greater chance of symptoms and admission to hospital.

In [ ]:
asthma_df = generate_sentinel_measure_combined(data_dict_practice, childs_table_dict["asthma"], "asthma", num_patients["asthma"], num_events["asthma"], codelist_link="https://www.opencodelists.org/codelist/opensafely/asthma-annual-review-qof/33eeb7da/")

In [ ]:
baseline, values, differences = calculate_statistics(asthma_df, '2019-04-01', ['2020-04-01', '2021-04-01'])
display_changes(baseline, values, differences, ['April 2020', 'April 2021'])
classify_changes(differences)

<a id="copd"></a>
### Chronic Obstructive Pulmonary Disease (COPD) Reviews

The codes used in for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/chronic-obstructive-pulmonary-disease-copd-review-qof/01cfd170/">this codelist</a>.  

<h3 class="details">What is it and why does it matter?</h3>

It is recommended by NICE that all individuals living with COPD have an annual review with the exception of individuals living with very severe (stage 4) COPD being reviewed at least twice a year.
If a patient has not been reviewed, it is possible that their COPD control may have worsened, leading to a greater chance of symptoms and admission to hospital.

In [ ]:
copd_df = generate_sentinel_measure_combined(data_dict_practice, childs_table_dict["copd"], "copd", num_patients["copd"], num_events["copd"], codelist_link="https://www.opencodelists.org/codelist/opensafely/chronic-obstructive-pulmonary-disease-copd-review-qof/01cfd170/")

In [ ]:
baseline, values, differences = calculate_statistics(copd_df, '2019-04-01', ['2020-04-01', '2021-04-01'])
display_changes(baseline, values, differences, ['April 2020', 'April 2021'])
classify_changes(differences)

<a id="med_review"></a>
### Medication Reviews

The codes used in for this measure are a combination of codes available in <a href="https://www.opencodelists.org/codelist/opensafely/care-planning-medication-review-simple-reference-set-nhs-digital/61b13c39/">this NHS Digitatil medication planning refset</a> and <a href="https://www.opencodelists.org/codelist/nhsd-primary-care-domain-refsets/medrvw_cod/20200812/">this primary care domain medication review refset</a>.
 
<h3 class="details">What is it and why does it matter?</h3>

Many medicines are used long-term and they should be reviewed regularly to ensure they are still safe, effective and appropriate.
Medication review is a broad term ranging from a notes-led review without a patient, to an in-depth Structured Medication Review with multiple appointments and follow-up. The codelist provided captures all types of reviews to give an overview of medication reviews in primary care.

In [ ]:
medication_review_df = generate_sentinel_measure_combined(data_dict_practice, childs_table_dict["medication_review"], "medication_review", num_patients["medication_review"], num_events["medication_review"], codelist_link="https://www.opencodelists.org/codelist/opensafely/care-planning-medication-review-simple-reference-set-nhs-digital/61b13c39/")

In [ ]:
baseline, values, differences = calculate_statistics(medication_review_df, '2019-04-01', ['2020-04-01', '2021-04-01'])
display_changes(baseline, values, differences, ['April 2020', 'April 2021'])
classify_changes(differences)

In [ ]:
def deciles_chart_subplots(
    df,
    period_column=None,
    column=None,
    title="",
    ylabel="",
    show_outer_percentiles=True,
    show_legend=True,
    ax=None,
):
    """period_column must be dates / datetimes"""
    sns.set_style("whitegrid", {"grid.color": ".9"})
    
    
    df = compute_deciles(df, period_column, column, show_outer_percentiles)
 
    linestyles = {
        "decile": {
            "line": "b--",
            "linewidth": 1,
            "label": "decile",
        },
        "median": {
            "line": "b-",
            "linewidth": 1.5,
            "label": "median",
        },
        "percentile": {
            "line": "b:",
            "linewidth": 0.8,
            "label": "1st-9th, 91st-99th percentile",
        },
    }
    label_seen = []
    for percentile in range(1, 100):  # plot each decile line
        data = df[df["percentile"] == percentile]
        add_label = False

        if percentile == 50:
            style = linestyles["median"]
            add_label = True
        elif show_outer_percentiles and (percentile < 10 or percentile > 90):
            style = linestyles["percentile"]
            if "percentile" not in label_seen:
                label_seen.append("percentile")
                add_label = True
        else:
            style = linestyles["decile"]
            if "decile" not in label_seen:
                label_seen.append("decile")
                add_label = True
        if add_label:
            label = style["label"]
        else:
            label = "_nolegend_"

        ax.plot(
            data[period_column],
            data[column],
            style["line"],
            linewidth=style["linewidth"],
            label=label,
        )
    ax.set_ylabel(ylabel, size=14)
    if title:
        ax.set_title(title, size=20)
    # set ymax across all subplots as largest value across dataset
    ax.set_ylim([0, df[column].max() * 1.05])
    ax.tick_params(labelsize=14)
    ax.set_xlim(
        [df[period_column].min(), df[period_column].max()]
    )  # set x axis range as full date range
    
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%B %Y"))
    if show_legend:
        ax.legend(
            bbox_to_anchor=(0.25, -0.8),  # arbitrary location in axes
            #  specified as (x0, y0, w, h)
            loc=CENTER_LEFT,  # which part of the bounding box should
            #  be placed at bbox_to_anchor
            ncol=1,  # number of columns in the legend
            fontsize=28,
            borderaxespad=0.0,
        )  # padding between the axes and legend
        #  specified in font-size units
    # rotates and right aligns the x labels, and moves the bottom of the
    # axes up to make room for them
    plt.gcf().autofmt_xdate()
    return plt

x = np.arange(0, 6, 1)
y = np.arange(0, 2, 1)

axs_list = [(i, j) for i in x for j in y]
fig, axs = plt.subplots(6, 2, figsize=(15,30), sharex='col')
fig.delaxes(axs[5, 1])

sentinel_measures = ["qrisk2", "asthma", "copd", "sodium", "cholesterol", "alt", "tsh", "rbc", 'hba1c', 'systolic_bp', 'medication_review']

titles = ['Cardiovascular disease risk assessment', 'Asthma review', 'COPD review', 'Renal assessment', 'Cholesterol testing', 'Liver function testing', 'Thyroid testing', 'Full blood count testing', 'Glycated haemoglobin testing', 'Blood pressure monitoring', 'Medication review']

for x, measure in enumerate(sentinel_measures):
    
    df = data_dict_practice[measure]
    
    
    
    if axs_list[x] == (4, 1):
        show_legend = True
    
    else:
        show_legend=False
        
        
       
    deciles_chart_subplots(df,
        period_column='date',
        column='rate',
        title=titles[x],
        ylabel="Rate per 1000",
        show_outer_percentiles=False,
        show_legend=show_legend,
        ax=axs[axs_list[x]])
 
fig.savefig('../backend_outputs/sentinel_measures_subplots.png')
plt.close()